In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
pic1 = Image.open("C:/Users/jared/OneDrive/Desktop/GPOE2025/tomography/easton.gif")
pic2 = Image.open("C:/Users/jared/OneDrive/Desktop/GPOE2025/tomography/westford.gif")

NameError: name 'Image' is not defined

In [ ]:
pic1

In [ ]:
pic2

In [ ]:
# tetsing with existing algorithms, have a hard time at limited angle!!
theta_max = 30
num_shots = 100

import numpy as np
import matplotlib.pyplot as plt

from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale

image = shepp_logan_phantom()
image = rescale(image, scale=0.4, mode='reflect', channel_axis=None)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5))

ax1.set_title("Original")
ax1.imshow(image, cmap=plt.cm.Greys_r)

theta = np.linspace(0.0, theta_max, num_shots, endpoint=False) #max(image.shape)
sinogram = radon(image, theta=theta)
dx, dy = 0.5 * theta_max / max(image.shape), 0.5 / sinogram.shape[0]
ax2.set_title("Radon transform\n(Sinogram)")
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax2.imshow(
    sinogram,
    cmap=plt.cm.Greys_r,
    extent=(-dx, theta_max + dx, -dy, sinogram.shape[0] + dy),
    aspect='auto',
)

fig.tight_layout()
plt.show()

In [ ]:
# choose a number of reconstruction iterations
iter_n = 160

from skimage.transform import iradon_sart

imkwargs = dict(vmin=-0.2, vmax=0.2)
reconstruction_sart = iradon_sart(sinogram, theta=theta)
error = reconstruction_sart - image
print(
    f'SART (1 iteration) rms reconstruction error: ' f'{np.sqrt(np.mean(error**2)):.3g}'
)

fig, axes = plt.subplots(2, 2, figsize=(8, 8.5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].set_title("Reconstruction\nSART")
ax[0].imshow(reconstruction_sart, cmap=plt.cm.Greys_r)

ax[1].set_title("Reconstruction error\nSART")
ax[1].imshow(reconstruction_sart - image, cmap=plt.cm.Greys_r, **imkwargs)

# Run a second iteration of SART by supplying the reconstruction
# from the first iteration as an initial estimate
errors = []
for i in range(iter_n-1):
    reconstruction_sart = iradon_sart(sinogram, theta=theta, image=reconstruction_sart)
    error = reconstruction_sart - image
    errors.append(np.sqrt(np.mean(error**2)))
    print(
        f'rms reconstruction error: '
        f'{np.sqrt(np.mean(error**2)):.3g}'
    )

ax[2].set_title("Reconstruction\nSART, "+str(iter_n)+" iterations")
ax[2].imshow(reconstruction_sart, cmap=plt.cm.Greys_r)

ax[3].set_title("Reconstruction error\nSART, "+str(iter_n)+" iterations")
ax[3].imshow(reconstruction_sart - image, cmap=plt.cm.Greys_r, **imkwargs)
plt.show()

In [ ]:
# this method doesn't seem to improve much with iteration. 
# Add in some metric of how much iteration helps, because the first one does the bulk
# Look also at how this depends on the number of projected images

# Take a look at where the error saturates to as a function of theta_max
plt.plot(np.linspace(1, iter_n,159),np.array(errors))


In [ ]:
# shitload of shots at various angle domains
thetas = [5,15,32.5,135,165,90,110]
errors = [0.1975,0.186,0.149, 0.0674,0.0308,0.105,0.089] #15deg is >160, 135 is also and 165
iter = [160,160,120,160,160,70,80]

plt.figure(1)
plt.plot(thetas,errors,'.')
plt.ylabel('RMS per-pixel error')
plt.xlabel('Field of view (FOV)')

plt.figure(2)
plt.plot(thetas,iter,'.')
plt.ylabel('ART iterations to convergence')
plt.xlabel('Field of view (FOV)')
plt.show()

In [ ]:
# Now look at how a 125deg shot depends on the number of images
shots125 = [100,30,10,3]
errors125 = [0.078,0.084,0.105,0.146]
iter125 = [160,160,160,160] #lim 160

plt.plot(shots125,errors125,'.',label='simulations')
plt.plot(np.linspace(1,100,100),0.07+0.07*1/(np.linspace(0,100,100))**(1/2))
plt.axhline(y=0.07,ls='dotted',c='r',label='125deg FOV limit')
plt.xscale('log')
plt.ylabel('RMS per-pixel error')
plt.xlabel('Number of projections')
plt.legend()
plt.show()

In [ ]:
# Now look at how a 30deg shot depends on the number of images
shots30 = [100,30,10,3]
errors30 = [0.151,0.158,0.169,0.188]
iter30 = [160,160,160,160] #lim 160

plt.plot(shots30,errors30,'.',label='simulations')
plt.plot(np.linspace(1,100,100),0.15+0.04*1/(np.linspace(0,100,100))**(1/2))
plt.axhline(y=0.149,ls='dotted',c='r',label='35deg FOV limit')
plt.xscale('log')
plt.ylabel('RMS per-pixel error')
plt.xlabel('Number of projections')
plt.legend()
plt.show()

In [ ]:
# these trends tell me we should make a sparse, wide array rather than a dense, narrow one!
# how does dimensionality factor in? 2D vs 3D recon

In [ ]:
# returning to aurora pics. Try pyabel?
import abel
original = abel.tools.analytical.SampleImage(name='Gerber').func
forward_abel = abel.Transform(original, direction='forward',
                              method='hansenlaw').transform
inverse_abel = abel.Transform(forward_abel, direction='inverse',
                              method='three_point').transform

fig, axs = plt.subplots(1, 2, figsize=(6, 3))

axs[0].imshow(forward_abel, clim=(0, None), cmap='ocean_r')
axs[1].imshow(inverse_abel, clim=(0, None), cmap='ocean_r')

axs[0].set_title('Forward Abel transform')
axs[1].set_title('Inverse Abel transform')

plt.tight_layout()
plt.show()


In [ ]:
plt.imshow(original)

In [ ]:
# How to make it not show just slices? Assumes cylindrical symetry... nvm

In [ ]:
# some potential libraries
# https://astra-toolbox.com/docs/geom3d.html
# https://tomopy.readthedocs.io/en/latest/ipynb/tomopy.html
# https://medium.com/@lotfi-habbiche/3d-model-reconstruction-from-images-implementing-with-opencv-3d1b370fa13c
# https://cvgl.stanford.edu/teaching/cs231a_winter1415/prev/projects/CS231a-FinalReport-sgmccann.pdf